# OAS->requests

In [1]:
#| default_exp oas_to_requests

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
import requests
from openapi_schema_pydantic import OpenAPI
import json
import yaml

In [4]:
#!wget https://petstore3.swagger.io/api/v3/openapi.json

In [5]:
#| export
BASE_URL = "https://petstore3.swagger.io/api/v3"

# Load and parse the OAS file
with open("openapi.json.3") as f:
    spec_dict = json.load(f)
oas = OpenAPI.parse_obj(spec_dict)

In [6]:
#| export
def get_request_body_from_schema(operation, oas):
    """Generates a default request body from the schema if `data` is not provided."""
    requestBody = operation.get("requestBody", {})
    if not requestBody: return None
    content = requestBody.get("content", {})
    schema_ref = content.get("application/json", {}).get("schema", {}).get("$ref")
    if schema_ref:
        schema_name = schema_ref.split("/")[-1]
        schema = oas.components.schemas.get(schema_name)
        if schema:
            return {"title": "Example Task", "completed": False}  # Example structure
    return None

In [7]:
#| export
def __generate_request_by_operation_id(oas, operation_id, parameters=None, data=None):
    # Search for the operation by `operationId`
    for path, path_item in oas.paths.items():
        for method, operation in path_item.dict().items():
            if operation and operation.get("operationId") == operation_id:
                url = f"{BASE_URL}{path}"
                headers = {"Content-Type": "application/json"}
                
                # Determine request body: use `data` if provided; otherwise, generate from schema
                request_body = data or get_request_body_from_schema(operation, oas)

                # Make the HTTP request dynamically
                response = requests.request(method.upper(), url, params=parameters, json=request_body, headers=headers)
                print(url, request_body)
                return response.json() if response.status_code in (200, 201) else response.status_code
    print(f"OperationId '{operation_id}' not found.")
    return None

In [11]:
#| export
def generate_request_by_operation_id(operation_id, parameters=None, data=None):
    return __generate_request_by_operation_id(oas, operation_id, parameters, data)

In [9]:
ans = generate_request_by_operation_id("findPetsByStatus", parameters={"status":"sold"})
ans[:3]

https://petstore3.swagger.io/api/v3/pet/findByStatus None


[{'id': 5081695936906578352,
  'category': {'id': 411886826470505861, 'name': ''},
  'name': "I4(u&blLw7Y~Xq:_A*6vU=;H]5R~#{BW-2[[#aI/q!'h*Y8jPg?NINmd+r",
  'photoUrls': ["neQ~Md/th'BL$*9OP.Ctd[~m68z`As{1Cf}e#T=Hz^*?v|sfAFat",
   '*v29XDy|TQ'],
  'tags': [{'id': 4695381958756277942,
    'name': "5Hb3X1h]ngm9'b!a^&!/T^DPM&6:D(c*F#d`%Fn3IVW!jkF5HI{d0qQj"},
   {'id': 3829192767566662587,
    'name': "!Bxsu/-ezDuPK/l'd])!Lepp4Kk:hjhd::/j~%T/y(;MmHFye8:MOs2[,4`"},
   {'id': 7415501824273923801, 'name': '%Ky97j'}],
  'status': 'sold'},
 {'id': 4987019593767351499,
  'category': {'id': 411886826470505861, 'name': ''},
  'name': "I4(u&blLw7Y~Xq:_A*6vU=;H]5R~#{BW-2[[#aI/q!'h*Y8jPg?NINmd+r",
  'photoUrls': ["neQ~Md/th'BL$*9OP.Ctd[~m68z`As{1Cf}e#T=Hz^*?v|sfAFat",
   '*v29XDy|TQ'],
  'tags': [{'id': 4695381958756277942,
    'name': "5Hb3X1h]ngm9'b!a^&!/T^DPM&6:D(c*F#d`%Fn3IVW!jkF5HI{d0qQj"},
   {'id': 3829192767566662587,
    'name': "!Bxsu/-ezDuPK/l'd])!Lepp4Kk:hjhd::/j~%T/y(;MmHFye8:MOs2[,4`"},


In [10]:
#| hide
import nbdev; nbdev.nbdev_export()